# Block Six Analytics Technical Challenge
### Bobby Shi
Thanks for the opportunity to go through the interview process for Block Six.  This was a really interesting challenge and I learned not only a lot about the basics of machine learning, but also simple stuff like how to make good code design and how to make my code run faster.  A lot of time was spent simply researching what a lot of the terms meant and understanding the nuances of the code.  All in all, it was a fantastic learning experience and something I really benefited from even beyond the scope of the interview process.

In [1]:
from collections import Counter
from IPython.display import display
from nltk.corpus import stopwords, wordnet
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from scipy.sparse import csr_matrix, hstack, lil_matrix
import numpy as np
import pandas as pd
import re

I import the packages I need.  I import the Counter object in order to obtain a tally of how many times each word appears in the tweets.  I import display simply so the dataframe in the final print looks pretty.  From nltk.corpus I import stopwords to be removed and wordnet as a lexicon of English words for a specific cleaning process (removing repeat letters from words).  I import PorterStemmer to stem words, but this will be an option that can be set.  word_tokenize is obviously to tokenize words.  From scipy.sparse I import csr_matrix and lil_matrix so I can create a matrix of 1.6 million rows and thousands of columns of zeros and ones (mostly zeros) without hogging all of my memory &mdash; this creates a sparse matrix; hstack is to append columns to the matrix.  Numpy is used for math, pandas is used to read data, and regex is used to clean tweets.

In [2]:
FILE = 'training.1600000.processed.noemoticon.csv'
WHITELIST = ['not']
MIN_OCCURENCES = 5
STEM = False
RATE = 0.001
EPOCHS = 2

These are some options that can be set.  FILE is obviously the data file.  WHITELIST indicates words that will not be removed that otherwise would be a stopword.  MIN_OCCURENCES is the minimum number of times a word must appear in all of the tweets in order to be counted in the regression.  STEM is whether or not to stem.  RATE represents the initial learning reate.  EPOCHS is how many times we will conduct a full pass through the data.

In [3]:
class Sentiment:
    
    def __init__(self, sentiments, tweets):
        '''
        Initialize the object with certain attributes
        '''
        
        # check if the number of tweets is equal to the number of sentiment scores
        if len(tweets) != len(sentiments):
            return -1
        
        self.tweets = tweets
        self.sentiments = sentiments
                
        self.wordlist = None
        
        self.stemmer = PorterStemmer()
                
        self.w = None
        
        print('Number of tweets: {}'.format(len(tweets)))
        
    @staticmethod
    def remove_re(tweet, regexp):
        '''
        Executes the basic cleaning methods (remove urls, numbers, symbols, etc.)
        '''
        tweet = re.sub(regexp, '', tweet)
        return tweet
        
    def tokenize(self, tweet, whitelist=WHITELIST, stem=STEM):
        '''
        Takes a tweet and cleans, tokenizes, removes stopwords, and stems
        '''
        
        # calls the static method to clean the Tweet
        tweet = Sentiment.remove_re(tweet, re.compile(r'\w+:\/\/\S+'))
        tweet = Sentiment.remove_re(tweet, re.compile(r'[^0-9A-Za-z \t]'))
        tweet = Sentiment.remove_re(tweet, re.compile(r'@[A-Za-z0-9]+'))
        tweet = Sentiment.remove_re(tweet, re.compile(r'^\d+\s|\s\d+\s|\s\d+$'))
       
        tweet = tweet.lower()
        
        # tokenize
        tokens = word_tokenize(tweet)
        
        # removes repeat letters from words and checks with dictionary to see if word exists
        for token in tokens:
            if re.search(r'(.)\1\1', token):
                token2 = re.sub(r'(.)\1+', r'\1\1', token)
                if wordnet.synsets(token2):
                    tokens[:] = [t.replace(token, token2) for t in tokens]
                else:
                    token3 = re.sub(r'(.)\1+', r'\1', token2)
                    tokens[:] = [t.replace(token, token3) for t in tokens]
            else:
                pass
            
        # remove stopwords
        stop = [word for word in stopwords.words('english') if word not in whitelist]
        tokens[:] = [token for token in tokens if token not in stop]
        
        if stem:
            # stem
            tokens[:] = [self.stemmer.stem(token) for token in tokens]
            return tokens
        else:
            return tokens
    
    def create_wordlist(self, min_occurences=MIN_OCCURENCES):
        '''
        Return wordlist if it already exists or create it
        '''
        if self.wordlist is not None:
            return self.wordlist
        
        else:
            all_words = Counter()
            
            for i in range(len(self.tweets)):
                tokens = self.tokenize(self.tweets[i])
                all_words.update(tokens)
                self.tweets[i] = tokens
            
            # only keep words that appear at least certain number of times
            list_of_words = [word for word, count in all_words.items() if count >= min_occurences]
            self.wordlist = {value: key for key, value in dict(enumerate(list_of_words)).items()}
            
            print('Number of words: {}'.format(len(self.wordlist)))
            print('\n')

            return self.wordlist
    
    def words_matrix(self):
        '''
        Creates a sparse matrix with row representing tweets and columns representing presence of words
        First column is all ones to indicate the bias term
        '''
        wordlist = self.create_wordlist()
        
        # creates a sparse matrix that can be sliced
        init_matrix = lil_matrix((len(self.tweets), len(wordlist)), dtype='int8')
        
        # populate matrix
        for i in range(len(self.tweets)):
            distinct_words = self.tweets[i]
            for word in distinct_words:
                if word in wordlist:
                    j = wordlist[word]
                    init_matrix[i,j] = 1
                else:
                    pass
        
        # add column of ones to indicate presence of bias
        ones = np.ones((len(self.tweets), 1), dtype='int8')
        
        # convert to csr matrix
        matrix = hstack([ones, init_matrix]).tocsr()
        
        n, m = matrix.shape
        print('The boolean matrix has {} rows and {} columns.'.format(n, m))
        print('\n')
        
        return matrix
    
    def optimize(self, initial_rate=RATE, epochs=EPOCHS):
        '''
        Set initial weights and execute the gradient descent process for the specified number of epochs
        '''
        # sparse matrix
        x = self.words_matrix()
        n, m = x.shape
        
        # sentiment scores - should match number of tweets
        y = np.array(self.sentiments)
        
        # set initial bias
        y_bar = np.mean(y)
        b_zero = np.exp(y_bar) / 1.0 + np.exp(y_bar)
        
        # set initial weight vector - bias followed by zeros
        self.w = np.append(np.array([b_zero]), np.zeros(m - 1))
         
        previous_cost = 0    
            
        # execute gradient descent
        for epoch in range(epochs):
            
            if epoch != 0:
                print('Shuffling...')
                print('\n')
            else:
                pass
                
            # shuffle dataset
            indices = np.random.permutation(n)
            x, y = x[indices,:], y[indices]
            
            print('Pass {} through data...'.format(epoch + 1))
            
            stochastic(self.w, x, y, initial_rate, epoch)
            
            cost_value = cost_function(self.w, x, y)
            
            print('Pass complete!')
            print('Cost value {}: {}'.format(str(epoch + 1), cost_value))
            
            if epoch != 0:
                print('Difference in cost: {}'.format(previous_cost - cost_value))
                print('\n')
            else:
                print('\n')
            
            previous_cost = cost_value
            
        
        return self.w

What I wanted to do with the Sentiment class was to create something reproducible.  Disregarding the names of the variables, everything done here can largely be applied to other forms of sentiment analysis with features other than simply the words.  For example, the cleaning methods are fairly adaptable, and one can choose which methods to use in accordance with the dataset.  The wordlist and the matrix are easily extendable to include whatever features one might want to use for analysis, and the optimization process can reflect whatever process is chosen (I learned there's a lot, lot more than just regression techniques).

In [4]:
def logistic(z):
    return 1.0 / (1.0 + np.exp(-1.0 * z))

In [5]:
def cost_function(w, x, y):
    '''
    w: weight vector
    x: tweet matrix
    y: sentiment vector
    '''
    n, m = x.shape
    z = x.dot(w)
    
    cost = (1.0 / n) * ((-1.0 * y).dot(np.log(logistic(z))) - (1.0 - y).dot(np.log(1.0 - logistic(z))))
    
    return cost

In [6]:
def stochastic(w, x, y, initial_rate, epoch):
    '''
    w: weight vector
    x: tweet matrix
    y: sentiment vector
    rate: learning rate
    '''
    n, m = x.shape
    rate = initial_rate
    
    for i in range(n):
        
        #adjust learning rate
        rate = learning_rate(initial_rate, i, epoch)
        
        z = x[i].dot(w)[0]
        nonzero = x[i].nonzero()[1]
        for j in nonzero:
            w[j] = w[j] - rate * (logistic(z) - y[i]) * x[i,j]

The previous three functions are all math.  The first one is the logistic function, represented by

$$p=f(z)=\frac{1}{1+e^{-z}}$$

where $p$ is the log odds of a tweet having positive sentiment.  In our analysis, we have that

$$z=b+\sum_{1}^n w_ix_i$$

where each $w_i$ indicates the weight of a certain word and $x_i$ indicates the presence of that word in the specific tweet ($b$ is the bias term).

The cost function given in the email is 

$$
C(\mathbf{w}\mid\mathbf{x}, y)=
\cases{
-\ln{\left(\frac{1}{1+e^{-\left(b+\sum_{1}^n w_ix_i\right)}}\right)} & y=1 \\
-\ln{\left(1-\frac{1}{1+e^{-\left(b+\sum_{1}^n w_ix_i\right)}}\right)} & y=0
}
$$

Of course, for ease, I will allow $z=b+\sum_{1}^n w_ix_i$.  Thus, for each tweet, we can simplify the above equation to be

$$
C(\mathbf{w}\mid\mathbf{x}, y)=-y\ln{\left(\frac{1}{1+e^{-\left(b+\sum_{1}^n w_ix_i\right)}}\right)}-(1-y)\ln{\left(1-\frac{1}{1+e^{-\left(b+\sum_{1}^n w_ix_i\right)}}\right)}.
$$

In both cases, $y=0$ and $y=1$, the cost functions are the same.  Now, replace $n$ with $m$; I want to do this to be consistent with how I wrote my code.  Thus, $m$ is the number of distinct words, and $n$ is the number of tweets &mdash; in our case $n=1,600,000$.  Thus, we can write the total mean cost as

$$
\frac{1}{n}\sum_{1}^{n} -y_j\ln{\left(\frac{1}{1+e^{-\left(b+\sum_{1}^m w_ix_i\right)}}\right)}-(1-y_j)\ln{\left(1-\frac{1}{1+e^{-\left(b+\sum_{1}^m w_ix_i\right)}}\right)}.
$$

Dot products are really useful here.  I choose to allow $b=w_0$, and $x_0$ is always $1$; thus, we have that

$$b+\sum_{1}^m w_ix_i=\sum_{0}^m w_ix_i.$$

This term is dependent on tweet; thus, it is the $j^{th}$ entry in a $n\times 1$ matrix.  This matrix is given by the dot product between our matrix $\mathbf{x}$, which is $n\times m$ (using the same $n$, $m$ as before), and the transpose of the weight matrix (vector) $\mathbf{w}$, which is $m\times 1$.  If we call this matrix $\mathbf{z}$, we have that

$$\mathbf{z}=\mathbf{x}\cdot \mathbf{w}^T.$$

Conveniently, numpy allows us to perform operations termwise, so we can do the logistic function on the entire matrix.  Now notice that each $y_j$ is the $j^{th}$ entry of the sentiment vector $\mathbf{y}$, which can be thought of as a $1\times n$ matrix.  Thus, we can again do dot product, and we have that the mean cost is

$$
-\frac{1}{n}(\mathbf{y}\cdot \ln{(\mathbf{z})}+(1-\mathbf{y})\cdot(\ln{(1-\mathbf{z})})).
$$

This is exactly what I coded into the function returning the cost value.  I call this function after each epoch to see if my cost value is decreasing after each pass.

The gradient descent process is also given in the email, which is

$$w_{\text{new}}\leftarrow w_{\text{old}} -\gamma(p-y)x_i.$$

Of course, I iterated by row on my $n\times m$ matrix.  To speed up my code, I simply returned the indices of nonzero values in each row, and then did the gradient descent step on those weights.  In this way, I didn't have to multiply by zero a bunch of times.

In [7]:
def learning_rate(initial_rate, i, epoch):
    '''
    Adjust the learning rate
    '''
    if epoch == 0:
        return initial_rate
    else:
        return initial_rate * (0.75)

Here I return with the learning rate, which is called in the gradient descent function.  This is the area that was the most murky to me, and I talked with professors and read a few papers on the subject &mdash; and it is the area that I would most like to learn more about.  In the end, the general idea I had was to at some point reduce the learning rate as I approach the bottom of my cost function.  Fortunately, log loss is convex, so I don't need to worry about local minimums, and so the reduction would lower the chances of any oscillations about or even potential divergence away from the global minimum.  I experimented with a lot of different rates, including letting it stay 0.001 the whole time, dividing by the number of tweets passed through after each step (so at the end, it would've been $\frac{0.001}{3.2\text{ million}}$, and making it bigger at 0.01 and making it smaller in the same way (dividing by number of tweets already passed through).  In the end, I let it stay at 0.001 through the first pass and then reduce it to 0.00075 during the second pass.

In [8]:
def main(file=FILE):
    
    print('Reading data...')
    
    # load file
    load = pd.read_csv(file, encoding="ISO-8859-1", header=None)
    
    # preprocess a bit
    load[0] = load[0].replace(to_replace=4, value=1)
    df = load[[0, 5]]
    df.columns = ['sentiment', 'tweet']
    
    print('Data loaded.')
    print('\n')
    print('Head and tail of data:')
    display(pd.concat([df.head(), df.tail()]))
    print('\n')
    
    # extract columns to lists
    sentiments = df['sentiment'].tolist()
    tweets = df['tweet'].tolist()
    
    # instantiate Sentiment object
    data = Sentiment(sentiments, tweets)
    w = data.optimize()
    
    # quickly reverse dictionary to be able to search by index
    reversed_wordlist = {value: key for key, value in data.wordlist.items()}
    
    # get 5 largest weights
    l_ind = np.argpartition(w, -5)[-5:]
    l_sort = l_ind[np.argsort(w[l_ind])][::-1]
    l_weights = w[l_sort]
    l_tuples = list(zip(l_sort, l_weights))
    
    print('Top 5 weights:')
    for tup in l_tuples:
        print('{}: {}'.format(reversed_wordlist[tup[0]], tup[1]))
    
    print('\n')
    
    # get 5 smallest weights
    s_ind = np.argpartition(w, 5)[:5]
    s_sort = s_ind[np.argsort(w[s_ind])]
    s_weights = w[s_sort]
    s_tuples = list(zip(s_sort, s_weights))
    
    print('Bottom 5 weights:')
    for tup in s_tuples:
        print('{}: {}'.format(reversed_wordlist[tup[0]], tup[1]))
    
    print('\n')
    
    # print bias term
    print('Bias: {}'.format(w[0]))

In the main function, I load data and preprocess it a bit.  The encoding is so I can read the file; without it resulted in an error.  I dropped all columns except for the sentiment values and the tweets, and for some reason positive sentiments were indicated by 4, so I changed all of it to 1.  Then I stored all the sentiment values and the tweets to lists.  I instantiate the Sentiment object and optimize it, returning weights, and then I print the top 5 weights, bottom 5 weights, and the bias.

In [9]:
main()

Reading data...
Data loaded.


Head and tail of data:


,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
1599995,1,Just woke up. Having no school is the best fee...
1599996,1,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,happy #charitytuesday @theNSPCC @SparksCharity...




Number of tweets: 1600000
Number of words: 86645


The boolean matrix has 1600000 rows and 86646 columns.


Pass 1 through data...
Pass complete!
Cost value 1: 0.5165943596188627


Shuffling...


Pass 2 through data...
Pass complete!
Cost value 2: 0.5041239183073967
Difference in cost: 0.012470441311466018


Top 5 weights:
letting: 1.7264024149960315
hows: 1.5696658721225454
carousella: 1.4464374513979492
till: 1.2807266501718593
hear: 1.2235996320188314


Bottom 5 weights:
ooh: -2.9092679938584163
premiere: -2.0320494453670266
aaw: -1.8920347712339727
spent: -1.8807150852187302
cameron: -1.7862225131754408


Bias: 0.27194235724608484


### Conclusions

One takeaway from this project was how important it was to speed up my code in any way possible, especially through vectorizing stuff, from creating the matrices to doing dot product instead of using a Python for loop.  Additionally, when first creating this, there were *a lot* of words that only appeared one time &mdash; like 500,000 of them.  Getting rid of those was absolutely vital in speeding up the process.

From my results, an interesting result was that negative words seemed to have beeh more negative than positive word were positive.  Additionally, no matter how I adjusted my learning rate, I never achieved a significant improvement (like on the order of 0.1) in cost, but this is probably by design.  This of course is somewhere I can improve.  Additionally, I can learn how to better clean and extract features &mdash; one area I wish I understood better is how to deal with negations, words like "not" and "no" especially.  For example, "love" is generally a positive word, but with "not" in front of it, like "I do not love that", the tweet becomes negative.  I also wish I could implement more features beyond just words.  All in all, it was a fantastic experience, and something I definitely fell in love with during the whole process.